In [1]:
import numpy as np
import gpstk
from Toolkit import *
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter('ignore', np.RankWarning)

In [2]:
def get_PRN_list(ofile,nfile):
    PRNs=[]
    oheader,odata=gpstk.readRinex3Obs(ofile,strict=True) 
    nheader,ndata=gpstk.readRinex3Nav(nfile)
    
    bcestore = gpstk.GPSEphemerisStore() 
    
    for ndato in ndata:
        ephem = ndato.toGPSEphemeris()
        bcestore.addEphemeris(ephem)
    bcestore.SearchNear() 
    for observation in odata:
        sats=[str(satID) for satID, datumList in observation.obs.iteritems() if str(satID).split()[0]=="GPS" ] 
        for sat in sats:
            if sat not in PRNs:
                PRNs.append(sat)
    return PRNs

def get_PRNsDATA(ofile,nfile):
    f1,f2=gpstk.L1_FREQ_GPS,gpstk.L2_FREQ_GPS
    alfa=1.0/((f1**2/f2**2)-1) 
    h=400.0e3 #the height of the ionospheric layer, which is assumed to be 400 km in this paper
    Re=6371.0e3 #Earth Radius
    gamma=gpstk.GAMMA_GPS
    PRNs=get_PRN_list(ofile,nfile)
    info_recv={}
    for PRN in PRNs:
        t,Icode,Iphase,__,ELEV,IPPS,L1,L2,C1,C2,Tgd=getdata(nfile,ofile,PRN)
        obs=get_arcs(t,Icode,Iphase,ELEV,IPPS,L1,L2,C1,C2) #gets arcs with Phase, Code, and adjusted times observations
        LevelArc={} #key:arc
        Time={}
        Zen={}
        for arc in range(len(obs)):
            time=obs[arc][0]
            Phase=obs[arc][1]
            Code=obs[arc][2]
            Elevation=obs[arc][3]

            #Phase on frequency L1,L2 station1
            PhaseL1=obs[arc][4]
            PhaseL2=obs[arc][5]
            CodeL1=obs[arc][6]
            CodeL2=obs[arc][7]
            #Detect data jumps on L=L1-L2
            jumps=datajump(Phase,2.5)

            #Divide in Sub-arcs using jumps
            miniarcs=sub_arcs(Phase,jumps)

            #Remove short arcs 
            toerase=[]
            for subarc in range(len(miniarcs)):
                if miniarcs[subarc].size<10:
                    toerase.append(subarc)
            miniarcs=np.delete(miniarcs,tuple(toerase))
        
            #Polinomial fit and Outlier Detection
            ##On each subarc check for slips and delete.
            pslips=[] #slips with polinomyal fit
            oslips=[] #slips oulier factor

            for i in miniarcs:
                __,pslip=poly_fit(Phase[i],time[i])
                pslips.append(pslip)
                __,oslip=outlier_detect(Phase[i],time[i]*3600)
                oslips.append(oslip)


            ##Using polinompyal fit and outlier factor methods we confirm slips
            confirmed=[] #a list of lists with the confirmed outliers on each subarc

            for i in miniarcs:
                outliers=confirmed_slip(time[i],Phase[i])
                confirmed.append(outliers)
            
            #############

            #Remove slips from both stations
            for i in range(len(confirmed)):
                if len(confirmed[i])!=0:
                    miniarcs[i]=np.delete(miniarcs[i],oslips[i])


            #Hatch Filter for code smoothing on each frequency before leveling
            #in every subarc
            newC1,newC2=CodeL1,CodeL2

            for i in miniarcs:
                newC1[i]=Smooth_code(CodeL1[i],PhaseL1[i]) #Codigo suavisado en L1 y L2
                newC2[i]=Smooth_code(CodeL2[i],PhaseL2[i])


            newC=(newC2-newC1)*alfa

            #Leveling Phase
            PhaseArc=[] #lists with subarcs leveled
            C=[]
            OldC=[]
            PhaseL1,PhaseL2=[],[]
            zenith=[]

            for i in miniarcs:
                Lfactor,new_IPHASE_arc=levelphase(newC[i],Phase[i],Elevation[i])
                PhaseArc.append(new_IPHASE_arc)  
                C.append(newC[i])
                OldC.append(Code[i])
                zenith.append(np.arcsin((Re*np.cos(Elevation[i])/(Re+h))))
                


            #Here all subarcs are merged into one single corrected continuous arc.
            new_IPHASE_arc=np.hstack(PhaseArc)
            C=np.hstack(C)
            OldC=np.hstack(OldC)
            zenith=np.hstack(zenith)
            
            ntimes=np.array([])

            for i in miniarcs:
                ntimes=np.append(ntimes,time[i])
                
            LevelArc[arc]=new_IPHASE_arc
            Time[arc]=ntimes
            Zen[arc]=zenith
        #LevelArc=np.hstack(LevelArc) #esto juntaría dos arcos separados! D:
        #Time=np.hstack(Time)
        #Zen=np.hstack(Zen)
        info_recv[PRN]=[Time,LevelArc,Tgd,Zen] #diccionario de diccs
    return info_recv



<h1>Receiver IFB</h1>


En este método se asume que la variación del vTEC (vertical ionospheric delays) de todos los satélites visibles en un instante es mínima cuando el ruido IFB(presente en el rinex y/o ionex) del satélite se remueve correctamente.

Podemos Despejar el IFB de receptor con la ecuación

$$vTEC=(TEC_{sl}-b_{s}-b_{r})Cos(\chi)$$
**Conocemos:**<br><br>
$TEC_{sl}$: Tec slant<br>
$\chi$: Zenith, $b_{s}$: Bias Satellite, incluido en rinex y/o ionex<br> 
<br><br>**Incognita:**<br><br>
$b_{r}$: Bias receptor

<img src="Receptor_IFB_Bias.png">

$\sigma_{t,u}$ es la sumatoria de todas las desviaciones en el día con un bías aleatorio. 


$\sigma_{u}(n)$ es la desviación de $VTEC$ en un instante $n$. Esto es sin errores de ambiguedad en la medida del delay en carrier.


$VTEC$ Vertical Total Electron Content medido por un solo satélite en tiempo $n$


In [3]:
def getIFB_Rec(ofile,nfile): #en cada epoca satelites que ve, L1,L2,P1,P2,Elevacion de cada satelite,IFB satelites (del rinex toca)
    sat_data=get_PRNsDATA(ofile,nfile)
    array={}
    for prn in sat_data.keys():
        data=sat_data[prn]
        arcos=sat_data[prn][0].keys()
        for arc in arcos: 
            times=data[0][arc]
            TEC_sl=data[1][arc]
            Tgd=data[2]
            zenith=data[3][arc]
            for t in range(len(times)):
                if times[t] not in array:
                    array[times[t]]=[[prn],[TEC_sl[t]],Tgd,[zenith[t]] ] #[PRN], [TEC_SLANT(PHASELEVEL)],Tgd,[Zenith]
                
                else:
                    array[times[t]][0].append(prn) #PRN
                    array[times[t]][1].append(TEC_sl[t]) #TEC slant
                    array[times[t]][3].append(zenith[t]) #zenith
    ############################################################################
    bias=np.arange(-30,30,0.1)*1e-9#600
    c=3e8
    vec_sum_desv=[] #vector with all sumdesv for every bias
    for bi in bias: #Cada bias candidato
        sumdesv_u=0 #sum of all standar desv  of that day with bias "bi"
        for n in array: 
            Mt=len(array[n][0]) #number of satellites observed at time n
            #print Mt, "Satelites en tiempo ",n
            TEC=[]
            for m in range(Mt):
                TECsl=array[n][1][m]
                tgd=array[n][2]
                zenith=array[n][3][m]
                vTEC=(TECsl-tgd-(bi*c))*np.cos(zenith)
                TEC.append(vTEC)
            desv_u=np.std(TEC) #sigma_u en tiempo n
            sumdesv_u=sumdesv_u+desv_u
        vec_sum_desv.append(sumdesv_u)
    best_bias=bias[np.argmin(vec_sum_desv)]
    return array,best_bias    

In [4]:
#ofile,nfile="conus_articulo/garf324.03.o","conus_articulo/garf324.03.n"

In [5]:
#print "Satellites in t=  0.0 seconds: ",array[0.0][0]
#print "Satellites in t=480.0 seconds: ",array[5400.0][0]

In [6]:
Stations=[
["conus_articulo/leba324.03.n","conus_articulo/leba324.03.o"],
["conus_articulo/colb324.03.n","conus_articulo/colb324.03.o"],
["conus_articulo/freo324.03.n","conus_articulo/freo324.03.o"],
["conus_articulo/mtvr324.03.n","conus_articulo/mtvr324.03.o"],
["conus_articulo/erla324.03.n","conus_articulo/erla324.03.o"],
["conus_articulo/woos324.03.n","conus_articulo/woos324.03.o"],
["conus_articulo/lsbn324.03.n","conus_articulo/lsbn324.03.o"],
["conus_articulo/grtn324.03.n","conus_articulo/grtn324.03.o"],
["conus_articulo/zob1324.03.n","conus_articulo/zob1324.03.o"]]

bias=[]

for station in Stations:
    nfile=station[0]
    ofile=station[1]
    name=ofile.split('/')[1][:4]
    __,bi=getIFB_Rec(ofile,nfile)
    bias.append((bi,name))
    
    

Poly:  4 Outlier factor:  4
Poly:  0 Outlier factor:  0
Poly:  5 Outlier factor:  5
Poly:  11 Outlier factor:  11
Poly:  31 Outlier factor:  31
Poly:  45 Outlier factor:  45


In [7]:
bias

[(-1.0499999999999723e-08, 'leba'),
 (-1.0699999999999727e-08, 'colb'),
 (-1.7599999999999823e-08, 'freo'),
 (-1.5199999999999792e-08, 'mtvr'),
 (2.990000000000085e-08, 'erla'),
 (-1.159999999999974e-08, 'woos'),
 (-2.3899999999999916e-08, 'lsbn'),
 (-2.5599999999999938e-08, 'grtn'),
 (-2.0999999999996036e-09, 'zob1')]